In [2]:
import re
import pandas as pd
import gensim.utils
from gensim.models import Phrases # for collocation detector for word2vec
import nltk
from gensim.parsing import preprocessing
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
from gensim.models import KeyedVectors

import sys 
sys.path.append('/Users/jessicahassibi/PycharmProjects/bachelor-thesis/src/data') # replace with your absolute path
# import modules from another folder
import WikiScraper
import data_analysis

New WikiScraper initialized.


In [3]:
persecuted_composers_path = "../data/verfolgte_komponisten_texte.json" # path to save wikipedia texts to or load from if file already existing
df = data_analysis.get_composers_dataframe(persecuted_composers_path)
df.head()

Texts have already been scraped to: ../data/verfolgte_komponisten_texte.json


,de_title,en_title,ar_title,fr_title,it_title,es_title,de_paragraphs,en_paragraphs,ar_paragraphs,fr_paragraphs,it_paragraphs,es_paragraphs,de_categories,en_categories,ar_categories,fr_categories,it_categories,es_categories
1,Abel Ehrlich,Abel Ehrlich,أبيل إرليخ,Abel Ehrlich,,,"[Ehrlich, aufgewachsen in einer deutschen jüdi...","[Erlich was born in 1915 in Cranz, East Prussi...",[أبيل إرليخ على موقع المكتبة المفتوحة (الإنجلي...,"[Erlich naît à Cranz, en Prusse-Orientale. En ...",,,"[Kategorie:Deutscher, Kategorie:Geboren 1915, ...","[Category:1915 births, Category:2003 deaths, C...","[تصنيف:أشخاص من زيلينوغرادسك, تصنيف:أشخاص من ش...",[Catégorie:Article de Wikipédia avec notice d'...,,
2,Admiral Horthy,Miklós Horthy,ميكلوش هورتي,Miklós Horthy,Miklós Horthy,Miklós Horthy,[Horthy stammte aus einer in der Großen Ungari...,[Miklós Horthy de Nagybánya was born at Kender...,[ميكلوش هورتي على موقع Encyclopædia Britannica...,"[Après la guerre, la république fédérative pop...",[Horthy nacque in una cittadina nell'attuale p...,[Pál Teleki Gyula Gömbös István Bethlen Primer...,"[Kategorie:Adliger, Kategorie:Autobiografie, K...","[Category:1868 births, Category:1957 deaths, C...","[تصنيف:أخطاء CS1: التاريخ, تصنيف:أشخاص مجريون ...","[Catégorie:Amiral autrichien, Catégorie:Anti-m...","[Categoria:Aggiungere sezioni - militari, Cate...","[Categoría:Anticomunistas de Hungría, Categorí..."
3,Alexander von Zemlinsky,Alexander von Zemlinsky,ألكسندر فون زيميلنسكي,Alexander von Zemlinsky,Alexander von Zemlinsky,Alexander von Zemlinsky,"[Stefan Schmidl: Zemlinsky, Alexander von. In:...",[Zemlinsky's best-known work is the Lyric Symp...,[ألكسندر فون زيميلنسكي على موقع Encyclopædia B...,[Zemlinsky étudie au conservatoire de Vienne o...,[Quintetto d'archi 1896 6 quartetti per archi ...,"[El padre de Alexander, Adolf, provenía de una...","[Kategorie:Alexander von Zemlinsky, Kategorie:...","[Category:1871 births, Category:1942 deaths, C...","[تصنيف:بذرة مؤلف نمساوي, تصنيف:بروتستانت نمساو...",[Catégorie:Article de Wikipédia avec notice d'...,"[Categoria:Austriaci del XIX secolo, Categoria...","[Categoría:Alexander Zemlinsky, Categoría:Comp..."
4,Alexandre Tansman,Alexandre Tansman,,Alexandre Tansman,Alexandre Tansman,Alexandre Tansman,[Tansman wurde in Łódź als Sohn jüdischer Elte...,[Tansman was born and raised in the Polish cit...,,[Né à Łódź le 11 juin 1897 dans une famille de...,"[Tansman nacque a Łódź, in Polonia, nel 1897. ...",[Sus primeros años transcurrieron en su Poloni...,"[Kategorie:Ehrendoktor einer Hochschule, Kateg...","[Category:1897 births, Category:1986 deaths, C...",,[Catégorie:Article de Wikipédia avec notice d'...,"[Categoria:BioBot, Categoria:Compositori class...",[Categoría:Compositores de música clásica de F...
5,Alfons Josef Biron,,,,,,[],,,,,,[],,,,,


In [26]:
####################################################################################################################################
####################################################################################################################################
# Methods to get Word Embeddings

In [4]:
####################################################################################################################################
# Using Word2Vec
# preprocessing the texts necessary
de_texts_processed = [[gensim.parsing.preprocess_string(text) for text in row] for row in df.de_paragraphs]
en_texts_processed = [[gensim.parsing.preprocess_string(text) for text in row] for row in df.en_paragraphs]
ar_texts_processed = [[gensim.parsing.preprocess_string(text) for text in row] for row in df.ar_paragraphs]
fr_texts_processed = [[gensim.parsing.preprocess_string(text) for text in row] for row in df.fr_paragraphs]
es_texts_processed = [[gensim.parsing.preprocess_string(text) for text in row] for row in df.es_paragraphs]
it_texts_processed = [[gensim.parsing.preprocess_string(text) for text in row] for row in df.it_paragraphs]
# preprocessed_texts_of_first_article = de_texts_processed[0])

all_sentences_processed = []
for lang in [de_texts_processed, en_texts_processed, ar_texts_processed, fr_texts_processed, es_texts_processed, it_texts_processed]:
    for document in lang:
        for sentence in document:
            all_sentences_processed.append(sentence)

In [5]:
s = "and some white space. Multiple sentences, too. This is nuts!"
gensim.parsing.preprocess_string(s)

['white', 'space', 'multipl', 'sentenc', 'nut']

In [6]:
# not so good
# Train a bigram detector.

bigram_transformer = Phrases(all_sentences_processed)

# Apply the trained MWE detector to a corpus, using the result to train a Word2vec model.
# Gensim implementiert Word2Vec-Algorithmus für das Training der Wortvektoren

model = gensim.models.Word2Vec(bigram_transformer[all_sentences_processed], min_count=2)

In [7]:
model = gensim.models.Word2Vec(all_sentences_processed, min_count=2)

In [8]:
model.train(all_sentences_processed, total_examples=model.corpus_count, epochs=model.epochs)

(1608882, 1836695)

In [9]:
model.build_vocab(all_sentences_processed, progress_per=10)


In [10]:
model.save("word2vec_multilingual.model")
#model = gensim.models.Word2Vec.load("word2vec_multilingual.model")

In [21]:
# get word vector
vector = model.wv['mahler']
print(vector)

[ 0.21207967  1.0662456   0.06637608 -0.28292155 -0.02998484 -0.9541701
  0.20160325  1.1216335  -0.37246817 -0.31035128  0.53176993 -0.47505438
  0.36975974 -0.13496822  0.78617966 -0.6063376  -0.03458015  0.04366869
 -0.27705777 -1.1747456  -0.08120186 -0.3732601   0.51007265 -0.08983222
 -0.8203365  -0.09593961 -0.24236378 -0.3362237  -0.2652088   0.06716047
  0.6126716  -0.5297314   0.6488223  -0.63756174  0.45199654  0.94105923
  0.61104226 -0.22461072 -0.0872469  -0.25759137 -0.21799055 -0.932162
 -0.06950531  0.13010553 -0.28288764 -0.5358332  -0.26055542  0.06355582
 -0.35385412 -0.30228022  0.2869221  -0.8884211  -0.36693695 -0.14855202
 -0.02921231  0.33783466  0.54890645  0.40721548 -0.6614451   0.06518238
 -0.11076296  0.44983155  0.13199599  0.31227365 -0.02776695  0.9021117
  0.5357478   0.33428234 -0.80864066  0.18585896  0.21384282 -0.04276929
  0.408708    0.36738867  0.8255888   0.7157498   0.17141286  0.97815543
 -0.41585457 -0.44754422 -0.8027532  -0.58328414  0.091

In [18]:
# find words used in similar context
model.wv.most_similar("composer")

KeyError: "Key 'composer' not present in vocabulary"

In [15]:
model.wv.distance("schönberg", "webern")

0.69894739985466

In [16]:
model.wv.similarity(w1="komponist", w2="nazi") 

0.6023786

In [17]:
model.wv.doesnt_match(['pianist', 'violinist', 'nazi'])

'pianist'

In [88]:
model.wv.most_similar(positive=["komponist", "jude"], negative=["schönberg"])

[('eher', 0.9178099632263184),
 ('französisch', 0.9147974252700806),
 ('politischen', 0.9130923748016357),
 ('dienst', 0.9129126667976379),
 ('geworden', 0.9105923771858215),
 ('krieg', 0.9105893969535828),
 ('damaligen', 0.9100316166877747),
 ('staatsbürgerschaft', 0.9097257256507874),
 ('verurteilt', 0.9095054268836975),
 ('bibliotheken', 0.9088475108146667)]